<a href="https://colab.research.google.com/github/BaLaurent/LLAMA-Tree-of-toughts-colab/blob/main/LLAMA_ToT_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a small experimentation based on the code from rhohndorf, you can find the original repository here:
https://github.com/rhohndorf/LLaMa-ToT/tree/main

If you have the paid version of Colab you might be able to use a bigger model, for the free version you'll need to stick to the 7B (or lower) version of your favorite model


In [ ]:
!pip install llama-cpp-python

In [ ]:
#@title Download model file
import requests

url = "https://huggingface.co/TheBloke/Wizard-Vicuna-7B-Uncensored-GGML/resolve/main/Wizard-Vicuna-7B-Uncensored.ggmlv3.q4_1.bin"#@param {type:"string"}
filename = "modelFile.bin"

response = requests.get(url)
response.raise_for_status()

with open(filename, "wb") as file:
    file.write(response.content)

print("File downloaded successfully!")


In [1]:
#@title LLM Setting
cfg_model_path = "/content/modelFile.bin"
cfg_temperature = .7
cfg_context_size = 2048

# Planning 
cfg_branching_factor = 3
cfg_max_plan_length = 3

In [2]:
#@title Templates
from string import Template

executer = Template("""
[Input] What year is it?
[Output] We are in [[DATE_YEAR]].
[Input] Please tell me the largest city in Europe.
[Output] The largest city in Europe is Moscow, the capital of Russia.
[Input] What can you tell me about Moscow?
[Output] Moscow, on the Moskva River in western Russia, is the nation's cosmopolitan capital. In its historic core is the Kremlin, a complex that's home to the president and tsarist treasures in the Armoury. Outside its walls is Red Square, Russia’s symbolic center.
[Input] What is a cat?
[Output] A cat is a domestic species of small carnivorous mammal. It is the only domesticated species in the family Felidae.
[Input] How do I pass command line arguments to a Node.js program?
[Output] The arguments are stored in process.argv.

    argv[0] is the path to the Node. js executable.
    argv[1] is the path to the script file.
    argv[2] is the first argument passed to the script.
    argv[3] is the second argument passed to the script and so on.
[Input] Name a color.
[Output] Blue.
[Input] What time is it?
[Output] It is [[DATE_TIME]].
[Input]
With the goal to $goal in mind, work on the following task:
$task
[Output]
""")

critic = Template("""
[Input]
Given the following incomplete plan to make an onlette
1) Gather all necessary ingredients, including eggs, butter or oil, salt and pepper, cheese (optional), and any other desired fillings such as vegetables or meat.
2) Crack the eggs into a mixing bowl and whisk them together until the yolks are broken up. 
3) Add in the butter or oil to the egg mixture and continue to whisk until it is well combined.
4) Season with salt and pepper to taste.

Analyze the following 3 statements in terms of correctness and plausibility as a next step in the above plan:
[1] Use a spatula to gently lift one side of the omelette up towards the center so that the uncooked eggs can flow underneath. Repeat this process every few minutes as you continue to cook the egg mixture until it is mostly set but still slightly runny on top.
[2] Heat a non-stick skillet over medium heat and add enough butter or oil to coat the bottom of the pan. 
[3] Once the butter has melted, pour in the egg mixture and let it cook for about 30 seconds until it starts to set on the edges.
Output the number of the best statement.
[Output]
2

[Input]
Given the following incomplete plan to $goal
$current_plan
Analyze the following $n_options statements in terms of correctness and plausibility as a next step in the above plan:
$potential_next_steps
Output the number of the best statement.
[Output]
""")

planner_long = Template("""
[Input]
GOAL: Learn to swim
PLAN: None. Please start with a first step!
Output the next valid step for the above PLAN. If the PLAN is complete output DONE.

[Output]
Buy a swim suit
    
[Input]
GOAL: Boil an egg
PLAN:
    1) Get a pot
    2) Fill pot with enough water to cover the egg
Output the next valid step for the above PLAN. If the PLAN is complete output DONE.
[Output]
Get a needle and carefully pierce the egg

[Input]
GOAL: Eat ice cream
PLAN:
    1) Buy ice cream
    2) Eat ice cream
Output the next valid step for the above PLAN. If the PLAN is complete output DONE.
[Output]
DONE

[Input]
GOAL: $goal
PLAN: $current_plan
Output the next valid step for the above PLAN. If the PLAN is complete output DONE.
[Output]
""")


def make_plan_str(plan_steps):
    if len(plan_steps) == 0:
        return  "None. Please start with a first step!"
    current_plan = '\n'
    for n, step in enumerate(plan_steps):
        current_plan += "    " + str(n+1) + ') ' + step.get("task") + '\n'
        # current_plan += '-------------------------------------------\n'
        # current_plan += step.get("result") + '\n'
    return current_plan 

def tpl_make_planner_prompt(goal, plan_steps):
    current_plan = make_plan_str(plan_steps)
    prompt = planner_long.substitute(goal=goal, current_plan=current_plan)
    return prompt

def tpl_make_critic_prompt(goal, plan_steps, potential_next_steps):
    current_plan = make_plan_str(plan_steps)
    next_steps_list = ''.join(['[' + str(i+1) + '] ' + step + '\n' for i,step in enumerate(potential_next_steps)])  
    prompt = critic.substitute(goal=goal, n_options=len(potential_next_steps), current_plan=current_plan, potential_next_steps=next_steps_list)
    return prompt

def make_executive_prompt(goal, plan, task):

    return executer.substitute(goal=goal, task=task)

In [ ]:
#@title Run ToT experiment
from llama_cpp import Llama

    
llm = Llama(cfg_model_path, n_ctx=2048)

def ask(prompt):
    response = llm(prompt, stop=["[Input]"], temperature=cfg_temperature, top_k=50)
    return response['choices'][0]['text'].rstrip().lstrip()


def solve(goal, plan):
    """Find a plan with DFS"""
    planner_prompt = tpl_make_planner_prompt(goal, plan)
    tmpPlanner = planner_prompt.replace("""
[Input]
GOAL: Learn to swim
PLAN: None. Please start with a first step!
Output the next valid step for the above PLAN. If the PLAN is complete output DONE.

[Output]
Buy a swim suit
    
[Input]
GOAL: Boil an egg
PLAN:
    1) Get a pot
    2) Fill pot with enough water to cover the egg
Output the next valid step for the above PLAN. If the PLAN is complete output DONE.
[Output]
Get a needle and carefully pierce the egg

[Input]
GOAL: Eat ice cream
PLAN:
    1) Buy ice cream
    2) Eat ice cream
Output the next valid step for the above PLAN. If the PLAN is complete output DONE.
[Output]
DONE""","")
    print(tmpPlanner)
    options = [ask(planner_prompt) for i in range(cfg_branching_factor)]
    while len(options) > 0:
        critic_prompt = tpl_make_critic_prompt(goal, plan, options)
        tmpCritic = critic_prompt.replace("""
[Input]
Given the following incomplete plan to make an onlette
1) Gather all necessary ingredients, including eggs, butter or oil, salt and pepper, cheese (optional), and any other desired fillings such as vegetables or meat.
2) Crack the eggs into a mixing bowl and whisk them together until the yolks are broken up. 
3) Add in the butter or oil to the egg mixture and continue to whisk until it is well combined.
4) Season with salt and pepper to taste.

Analyze the following 3 statements in terms of correctness and plausibility as a next step in the above plan:
[1] Use a spatula to gently lift one side of the omelette up towards the center so that the uncooked eggs can flow underneath. Repeat this process every few minutes as you continue to cook the egg mixture until it is mostly set but still slightly runny on top.
[2] Heat a non-stick skillet over medium heat and add enough butter or oil to coat the bottom of the pan. 
[3] Once the butter has melted, pour in the egg mixture and let it cook for about 30 seconds until it starts to set on the edges.
Output the number of the best statement.
[Output]
2
""","")
        print(tmpCritic)
        chosen_option = options.pop(int(ask(critic_prompt)) -1)

        # Termination criterion 1: Solution found. The plan is complete and can be returned 
        if chosen_option == "DONE":
            return plan

        # Termination criterion 2: Max plan length reached and no valid plan found. Backtrack 1 step.
        if len(plan) == cfg_max_plan_length:
            return None
        
        # Explore subtree
        valid_plan =  solve(goal, plan+[{'task':chosen_option, 'state':None}])
        if valid_plan != None:
            return valid_plan
        


if __name__ == "__main__":
    # goal = "solve for x,y and z in the following system of equations x=17z + 3; y=2x; z=1"
    goal = "Earn money online without finding clients"#@param {type:"string"}
    # goal = "Combine the numbers 4,9,10 and 13 with the basic arithmatic operations(+,-,*,/) to obtain the result 24"
    plan = None

    while plan == None:
        plan = solve(goal, [])

    print(plan)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 




[Input]
GOAL: Earn money online without finding clients
PLAN: None. Please start with a first step!
Output the next valid step for the above PLAN. If the PLAN is complete output DONE.
[Output]



Llama.generate: prefix-match hit
Llama.generate: prefix-match hit



[Input]
Given the following incomplete plan to Earn money online without finding clients
None. Please start with a first step!
Analyze the following 3 statements in terms of correctness and plausibility as a next step in the above plan:
[1] Buy a domain name and hosting plan
[2] Start by creating a website or blog and writing about topics that interest you.
[3] DONE

Output the number of the best statement.
[Output]



Llama.generate: prefix-match hit
